In [ ]:
### Chargement des Packages et utilitaires nécessaires 
# !pip install -q gcsfs shap optuna graphviz lightgbm

import optuna
import pandas as pd, numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from graphviz import Source
from IPython.display import SVG
%matplotlib inline

In [ ]:
### Importation de la base de données 
df_lait=pd.read_csv('BD_COUT_LAIT_STATS_NEW_3POP_CONTROL.csv',sep=';')
df_lait.head()

In [ ]:
### Liste des colonnes 
print(df_lait.columns.tolist())

In [ ]:
df_lait.describe()

In [ ]:
### Sélection des variables

df_lait=df_lait[['DEP', 'IDQ', 'v2_1', 'SYS2', 'Prop_produit_lait', 'ANNEE_Prod', 'NBR_TRIM3', 'NBR_TRIM4', 'NBR_TRIM1', 'SFP', 'NBR_VL', 'LAIT_VENDU', 'NBR_UMO_EXPT',
                'NBR_UMO_SAL', 'LAIT_VENDU_UMOREM','SALAIRE','FRAIS_DIVERS','BATIMENT_INST','FONCIER_CAPITAL', 'FRAIS_FINANCIERS', 'REM_CAPITAUX_PROPRE','MECA','FRAIS_ELEVAGE', 'APPROV_SURFACE', 'APPROV_ANIMAUX','TOT_AMORT','PRODUIT_LAIT', 'PRODUITS_JOINTS',
                'REM_PAR_PRODUIT_SMIC']]

In [ ]:
df_lait['v2_1']=df_lait['v2_1'].astype(int)

In [ ]:
### Définition du positionnement 
#if pd.notnull(df_lait['SFP']): 
    
df_lait['LAIT_VENDU_SFP']=df_lait['LAIT_VENDU']/df_lait['SFP']

df_lait['LAIT_VENDU_NB_VL']=df_lait['LAIT_VENDU']/df_lait['NBR_VL']

df_lait['Prix_lait']=(df_lait['PRODUIT_LAIT']*1000/df_lait['LAIT_VENDU'])*1000

df_lait.replace([np.inf, -np.inf], np.nan, inplace=True)
df_lait=df_lait.fillna(0)

### Variables dummies montagne et plaine. 
#cat_names = {1:'Plaine', 3:'Montagne'}
#for elem in df_lait['v2_1'].unique():
    #df_lait[cat_names[elem]] = df_lait['v2_1'] == elem

In [ ]:
df_lait[df_lait['SFP']==0].head()

In [ ]:
df_lait['Prix_lait'].describe()

In [ ]:
df_lait['LAIT_VENDU'].describe()

In [ ]:
df_lait['PRODUIT_LAIT'].describe()

In [ ]:
df_lait.shape

In [ ]:
df_lait=df_lait.rename(columns={'DEP':'ID: departement','IDQ':'ID: dossier','ANNEE_Prod':'ID: annee',
                               'FRAIS_DIVERS':'Pratiques_elevage: frais_divers','Prop_produit_lait':'Pratiques_elevage: taux_specialisation','Prop_produit_lait':'Pratiques_elevage: taux_specialisation',
                               'FRAIS_ELEVAGE':'Pratiques_elevage: frais_elevage','APPROV_SURFACE':'Pratiques_elevage: approvisionnement_surfaces','APPROV_ANIMAUX':'Pratiques_elevage: approvisionnement_animaux',
                               'PRODUIT_LAIT':'Pratiques_elevage: produit_lait','PRODUIT_JOINTS':'Pratiques_elevage: produit_joints',
                               'FONCIER_CAPITAL':'Dynamique_structurelle: foncier_capital','FRAIS_FINANCIERS':'Dynamique_structurelle: frais_financiers','REM_CAPITAUX_PROPRE':'Dynamique_structurelle: remunaration_capitaux_propres',
                               'SALAIRE':'Dynamique_structurelle: salaires','LAIT_VENDU_UMOREM':'Dynamique_structurelle: lait_vendu_umo_remunere','MECA':'Dynamique_structurelle: mecanisation',
                               'BATIMENT_INST':'Dynamique_structurelle: batiment_installation','TOT_AMORT':'Dynamique_structurelle: total_amortissement',
                               'NBR_UMO_EXPT':'Dimension: umo_exploitant','NBR_VL':'Dimension: taille_troupeau','SFP':'Dimension: surface_fouragere',
                               'LAIT_VENDU_NB_VL':'Positionnement: lait_vendu_par_vl','LAIT_VENDU_SFP':'Positionnement: lait_vendu_par_sfp','v2_1':'Invariant: montagne_plaine','SYS2':'Bio: bio_conventionnel',
                               'NBR_TRIM3':'Invariant: du_trimestre_3','NBR_TRIM4':'Invariant: du_trimestre_4', 'NBR_TRIM1':'Invariant: du_trimestre_1',
                               'REM_PAR_PRODUIT_SMIC':'Cible: remuneration_par_produit_smic'})

In [ ]:
orders=['ID','Invariant','Bio','Dimension','Dynamique_structurelle','Pratiques_elevage','Positionnement','Cible']
df_lait=pd.concat([df_lait.filter(regex=f'{order}:') for order in orders],1)

In [ ]:
df_lait.head()

In [ ]:
#df_lait.shape
df_lait=df_lait[df_lait['Positionnement: lait_vendu_par_sfp']<20000]

In [ ]:
df_lait.info()

In [ ]:
df_lait.describe(include=float)

In [ ]:
df_lait=df_lait[df_lait['Pratiques_elevage: frais_elevage']>0]
df_lait.shape

In [ ]:
#dessine la graphe d'Elbow pour trouver le nombre idéal de clusters 
def findbestclusters(data,maxclusters=30,minclusters=2):
  Sum_of_squared_distances = []
  K = range(minclusters,maxclusters)
  for k in K:
      km = KMeans(n_clusters=k, random_state=49)
      km = km.fit(data)
      Sum_of_squared_distances.append(km.inertia_)
  plt.plot(K, Sum_of_squared_distances, 'bx-')
  plt.xlabel('k')
  plt.ylabel('Sum_of_squared_distances')
  plt.title('Elbow Method For Optimal k')
  plt.show()

  sil = []
  # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
  for k in range(minclusters+1, maxclusters+1):
    kmeans = KMeans(n_clusters = k).fit(data)
    labels = kmeans.labels_
    sil.append({'k':k,'score':silhouette_score(data, labels, metric = 'euclidean')})
  sil=pd.DataFrame(sil)
  sil.plot(x='k',y='score')
  return sil[sil.score==sil.score.max()].iloc[0,0]


In [ ]:
### Le clustering 
n_clusters=findbestclusters(df_lait.filter(regex='Positionnement'),maxclusters=10,minclusters=1)

In [ ]:
n_clusters=5
km = KMeans(n_clusters=n_clusters, random_state=49)
y=km.fit_predict(df_lait.filter(regex='Positionnement'))
fig, ax = plt.subplots(figsize=(10,8)) 
sns.scatterplot(data=df_lait, x='Positionnement: lait_vendu_par_sfp',y='Positionnement: lait_vendu_par_vl',hue=y,ax=ax,palette="Set2",)

In [ ]:
dfclustered=df_lait.copy()
dfclustered['cluster']=y
dfclustered.to_csv('dfclustered.csv')

In [ ]:
dfclustered.head()

In [ ]:
import plotly.express as px
fig = px.scatter_3d(dfclustered, x='Positionnement: lait_vendu_par_vl',y='Positionnement: lait_vendu_par_sfp', z='Cible: remuneration_par_produit_smic',
              color='cluster')
fig.show("notebook")

In [ ]:
#détermination des critères de clusters
sns.countplot(x='cluster',data=dfclustered, palette='Set2')

In [ ]:
for c in dfclustered.columns:
  fig, ax = plt.subplots(figsize=(4,4)) 
  sns.boxplot(x="cluster", y=c, data=dfclustered,
                   ax=ax, palette='Set2')

In [ ]:
### Getting the dummies of Clusters 
cat_names = {0:'Cluster0', 1:'Cluster1',2:'Cluster2',3:'Cluster3',4:'Cluster4'}
for elem in dfclustered['cluster'].unique():
    dfclustered[cat_names[elem]] = dfclustered['cluster'] == elem
    
dfclustered.head()

In [ ]:
import plotly.graph_objects as go
values=dfclustered.groupby('cluster').mean()

values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
for cluster in dfclustered.cluster.unique():
  print(cluster)
  cluster0=dfclustered.copy()
  cluster0['cluster']=cluster0['cluster']==cluster
  cluster0=cluster0.drop(['ID: departement','ID: dossier','ID: annee'],1)
  X=cluster0.drop('cluster',1)
  X=cluster0.filter(regex='Positionnement')
  y=cluster0.cluster
  estimator = DecisionTreeClassifier(max_depth=3)
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
  estimator.fit(X_train, y_train)
  estimator.predict(X_test)
  print(estimator.score(X_test, y_test))
  graph = Source(tree.export_graphviz(estimator, out_file=None
    , feature_names=X.columns
    , class_names=True
    , filled = True))
  display(SVG(graph.pipe(format='svg')))

In [ ]:
### Caractérisations de ces classes par rapport à leurs pratiques d'élevages 
for cluster in dfclustered.cluster.unique():
  print(cluster)
  cluster0=dfclustered.copy()
  cluster0['cluster']=cluster0['cluster']==cluster
  cluster0=cluster0.drop(['ID: departement','ID: dossier','ID: annee'],1)
  X=cluster0.drop('cluster',1)
  X=cluster0.filter(regex='Pratiques_elevage')
  y=cluster0.cluster
  estimator = DecisionTreeClassifier(max_depth=3)
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
  estimator.fit(X_train, y_train)
  estimator.predict(X_test)
  print(estimator.score(X_test, y_test))
  graph = Source(tree.export_graphviz(estimator, out_file=None
    , feature_names=X.columns
    , class_names=True
    , filled = True))
  display(SVG(graph.pipe(format='svg')))

In [ ]:
###### Caractérisation des types de pratiques d'élevages
## Passons d'abord certaines les  variables de positionnement en pratiques d'élevage. 
df_lait2=df_lait.rename(columns={'Positionnement: lait_vendu_par_vl':'Pratiques_elevage: lait_vendu_par_vl',
       'Positionnement: lait_vendu_par_sfp':'Pratiques_elevage: lait_vendu_par_sfp'})

df_lait2['Pratiques_elevage: charges_elevage']= df_lait2['Pratiques_elevage: frais_divers'] + df_lait2['Pratiques_elevage: frais_elevage'] + df_lait2['Pratiques_elevage: approvisionnement_surfaces'] + df_lait2['Pratiques_elevage: approvisionnement_animaux']

df_lait2=df_lait2.drop(columns=['Pratiques_elevage: frais_divers','Pratiques_elevage: frais_elevage','Pratiques_elevage: approvisionnement_surfaces','Pratiques_elevage: approvisionnement_animaux'])
orders=['ID','Invariant','Bio','Dimension','Dynamique_structurelle','Pratiques_elevage','Positionnement','Cible']
df_lait2=pd.concat([df_lait2.filter(regex=f'{order}:') for order in orders],1)
df_lait2.head()

In [ ]:
df_lait.head()

In [ ]:
## Clustering des pratiques d'élevages 
n_clusters2=findbestclusters(df_lait2.filter(regex='Pratiques_elevage'),maxclusters=15,minclusters=1)

In [ ]:
n_clusters=6
km = KMeans(n_clusters=n_clusters, random_state=49)
y=km.fit_predict(df_lait2.filter(regex='Pratiques_elevage'))
fig, ax = plt.subplots(figsize=(10,8)) 
sns.scatterplot(data=df_lait2, x='Pratiques_elevage: charges_elevage',y='Pratiques_elevage: lait_vendu_par_vl',hue=y,ax=ax,palette="Set2",)

In [ ]:
dfclustered2=df_lait2.copy()
dfclustered2['cluster']=y
dfclustered2.to_csv('dfclustered2.csv')

In [ ]:
dfclustered2.head()

In [ ]:
### Caractérisations de ces classes par rapport à leurs pratiques d'élevages 
for cluster in dfclustered2.cluster.unique():
  print(cluster)
  cluster0=dfclustered2.copy()
  cluster0['cluster']=cluster0['cluster']==cluster
  cluster0=cluster0.drop(['ID: departement','ID: dossier','ID: annee'],1)
  X=cluster0.drop('cluster',1)
  X=cluster0.filter(regex='Pratiques_elevage')
  y=cluster0.cluster
  estimator = DecisionTreeClassifier(max_depth=3)
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
  estimator.fit(X_train, y_train)
  estimator.predict(X_test)
  print(estimator.score(X_test, y_test))
  graph = Source(tree.export_graphviz(estimator, out_file=None
    , feature_names=X.columns
    , class_names=True
    , filled = True))
  display(SVG(graph.pipe(format='svg')))

In [ ]:
dfclustered[dfclustered['ID: departement']==76].head()

In [ ]:
dfclustered.info()

In [ ]:
### Enlever les variables de positonnement 
df_lait_cf=dfclustered.drop(columns=['Positionnement: lait_vendu_par_sfp','Positionnement: lait_vendu_par_vl'],axis=1)
df_lait_cf=dfclustered.drop(columns=['ID: departement','ID: dossier','ID: annee'],axis=1) 

### Recodage de quelques variables 

df_lait_cf['Invariant: du_trimestre_3'] = np.where(df_lait_cf['Invariant: du_trimestre_3'] == 1, "oui", "non")
df_lait_cf['Invariant: du_trimestre_4'] = np.where(df_lait_cf['Invariant: du_trimestre_4'] == 1, "oui", "non")
df_lait_cf['Invariant: du_trimestre_1'] = np.where(df_lait_cf['Invariant: du_trimestre_1'] == 1, "oui", "non")
df_lait_cf['Invariant: montagne_plaine'] = np.where(df_lait_cf['Invariant: montagne_plaine'] == 1, "plaine", "montagne")
df_lait_cf['Bio: bio_conventionnel'] = np.where(df_lait_cf['Bio: bio_conventionnel'] == 1, "oui", "non")

df_lait_cf.loc[df_lait_cf['cluster'] == 0, 'cluser_label'] = 'cluster_0'
df_lait_cf.loc[df_lait_cf['cluster'] == 1, 'cluser_label'] = 'cluster_1'
df_lait_cf.loc[df_lait_cf['cluster'] == 2, 'cluser_label'] = 'cluster_2'
df_lait_cf.loc[df_lait_cf['cluster'] == 3, 'cluser_label'] = 'cluster_3'
df_lait_cf.loc[df_lait_cf['cluster'] == 4, 'cluser_label'] = 'cluster_4'

df_lait_cf.info()

In [ ]:
df_lait_cf=df_lait_cf.drop(columns=['cluster'],axis=1) 
df_lait_cf.head()

In [ ]:
##### Suite des analyses, prédiction de la variables cibles rémunération permise par la rémunération permise par le produit. Package de Microsoft. DiCE. 
import dice_ml
from dice_ml import Dice
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
outcome_name='Cible: remuneration_par_produit_smic'

In [ ]:
df_lait_cf.info()

In [ ]:
df_lait_cf=df_lait_cf.drop(columns=['Positionnement: lait_vendu_par_sfp','Positionnement: lait_vendu_par_vl'],axis=1)

In [ ]:
df_lait_cf=df_lait_cf.drop(columns=['Pratiques_elevage: frais_divers'],axis=1)

In [ ]:
df_lait_cf.info()

In [ ]:
#df_lait_cf['cluster']=df_lait_cf['cluster'].astype('category')
#df_lait_cf.info()

In [ ]:
df_lait_cf.head()

In [ ]:
target=dfclustered[outcome_name]
continuous_features1 = df_lait_cf.filter(regex='Dimension').columns.tolist()
continuous_features2 = df_lait_cf.filter(regex='Dynamique_structurelle').columns.tolist()
continuous_features3 = df_lait_cf.filter(regex='Pratiques_elevage').columns.tolist()
continuous_features_df_lait_cf=continuous_features1+continuous_features2+continuous_features3
continuous_features_df_lait_cf

In [ ]:
target

In [ ]:
### Density plots
sns.distplot(df_lait_cf['Cible: remuneration_par_produit_smic'], hist=True, kde=True, 
                 bins=int(180/5), color = 'darkblue', 
                 hist_kws={'edgecolor':'black'},
                 kde_kws={'linewidth': 4})

In [ ]:
plt.plot(df_lait_cf['Dimension: umo_exploitant'], df_lait_cf['Cible: remuneration_par_produit_smic'], 'o', color='blue')

In [ ]:
df_lait_cf['cluster']=df_lait_cf['cluster'].astype('int')

categorical_features=['cluster']
categorical_features

In [ ]:
 %%time
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import HuberRegressor, RANSACRegressor

# Split data into train and test
datasetX = df_lait_cf.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=21)
#categorical_features=x_train.columns.difference(numerical) 
categorical_features = x_train.columns.difference(continuous_features_df_lait_cf) 
# We create the preprocessing pipelines for both numeric and categorical data.

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(transformers=[('num', numeric_transformer, continuous_features_df_lait_cf),('cat', categorical_transformer, categorical_features)])

reg_lait = Pipeline(steps=[('preprocessor', transformations),
                           ('regressor', MLPRegressor(hidden_layer_sizes=(400,500,), activation="relu" ,random_state=1))])
model_lait=reg_lait.fit(x_train, y_train)
reg_lait.score(x_test, y_test)

In [ ]:
#### categorical_features

In [ ]:
from sklearn import metrics
y_pred = reg_lait.predict(x_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
dfclustered_new=dfclustered.copy()

cat_names = {0:'Cluster0', 1:'Cluster1',2:'Cluster2',3:'Cluster3',4:'Cluster4'}
for elem in dfclustered_new['cluster'].unique():
    dfclustered_new[cat_names[elem]] = dfclustered_new['cluster'] == elem
    #dfclustered_new[cat_names[elem]].astype(int)

dfclustered_new['Cluster0']=dfclustered_new['Cluster0'].astype(int)
dfclustered_new['Cluster1']=dfclustered_new['Cluster1'].astype(int)
dfclustered_new['Cluster2']=dfclustered_new['Cluster2'].astype(int)
dfclustered_new['Cluster3']=dfclustered_new['Cluster3'].astype(int)
dfclustered_new['Cluster4']=dfclustered_new['Cluster4'].astype(int)

dfclustered_new.head()

In [ ]:
dfclustered_new.info()

In [ ]:
import lightgbm as lgb
random_state = 7
X = dfclustered_new.drop(['Cible: remuneration_par_produit_smic','cluster','ID: departement','ID: dossier','ID: annee','Positionnement: lait_vendu_par_sfp','Positionnement: lait_vendu_par_vl'],axis=1)
y=dfclustered_new['Cible: remuneration_par_produit_smic']

X=X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
model=lgb.LGBMRegressor(n_estimators=50000)
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20,verbose=False)
model.score(X_test,y_test)

In [ ]:
df.head()

In [ ]:
from sklearn import metrics
y_pred = model.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
outcome_name

In [ ]:
df_lait_cf.info()

In [ ]:
### Counterfactual explanation 
d_lait = dice_ml.Data(dataframe=df_lait_cf, continuous_features=continuous_features_df_lait_cf, outcome_name=outcome_name)
# We provide the type of model as a parameter (model_type)
m_lait = dice_ml.Model(model=model_lait, backend="sklearn", model_type='regressor')

In [ ]:
exp_genetic_lait = Dice(d_lait, m_lait, method="genetic")

In [ ]:
import re

r1 = re.compile('Pratiques_elevage*')
r2 = re.compile('Dynamique_structurelle*')
r3 = re.compile('Dimension*')
r4 = re.compile('Invariant*')
r5 = re.compile('Bio*')
r6 = re.compile('cluster*')
features_vary=list(filter(r1.match, continuous_features_df_lait_cf)) + list(filter(r2.match, continuous_features_df_lait_cf)) + list(filter(r3.match, continuous_features_df_lait_cf)) + list(filter(r5.match, continuous_features_df_lait_cf))
query_instances_lait = datasetX[2:3]
genetic_lait = exp_genetic_lait.generate_counterfactuals(query_instances_lait, 
                                                             total_CFs=3, 
                                                             features_to_vary=features_vary,
                                                             proximity_weight=2.5, diversity_weight=1,
                                                             desired_range=[1,2])
genetic_lait.visualize_as_dataframe(show_only_changes=True)

In [ ]:
exp=genetic_lait.cf_examples_list[0].final_cfs_df

In [ ]:
exp.head()